PARTITE 

In [ ]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# Rimuovo import specifico renderer, Plotly dovrebbe rilevarlo
# import plotly.io as pio
# pio.renderers.default = "vscode"
from ipywidgets import widgets, Layout, VBox
from IPython.display import display, clear_output
from typing import List, Dict, Callable, Optional, Any
import numpy as np
import traceback # Per stampa errori dettagliata

print("--- Inizio Script Analisi Partite ---")

# --- Costanti per nomi colonne ---
COL_SEASON = 'Season'
COL_DATE = 'Date'       # Formato atteso: Es. 30/07/1999, 2000-08-20, etc.
COL_HT = 'HomeTeam'
COL_AT = 'AwayTeam'
COL_FTHG = 'FTHG'       # Full Time Home Goals
COL_FTAG = 'FTAG'       # Full Time Away Goals
COL_FTR = 'FTR'         # Full Time Result (H=Home Win, D=Draw, A=Away Win)
# Potrebbero esserci altre colonne come HTHG, HTAG, Referee, B365H, etc. ma non sono usate nei grafici di base

# Colonne derivate
COL_TOTAL_GOALS = 'TotalGoals'
COL_GOAL_DIFF = 'GoalDifference' # FTHG - FTAG
COL_PTS_H = 'PointsH'
COL_PTS_A = 'PointsA'
COL_SCORE = 'Score' # Es. '2-1'
COL_YEAR = 'Year' # Aggiunto per rolling average

# --- Funzioni di Utilità ---

def load_data(directory: str) -> pd.DataFrame:
    """Carica e combina dati da file CSV 'season*.csv' in una directory.
       Aggiunge colonna 'Season', gestisce errori comuni e tipi di dati base.
    """
    all_data: List[pd.DataFrame] = []
    # Colonne minime richieste per la maggior parte dei grafici
    required_core_columns = [COL_HT, COL_AT, COL_FTHG, COL_FTAG, COL_FTR]

    print(f"Inizio caricamento dati da: {directory}")
    if not os.path.isdir(directory):
        print(f"ERRORE CRITICO: La directory specificata '{directory}' non esiste o non è una directory.")
        return pd.DataFrame()

    files_processed = 0
    files_found = 0
    for filename in sorted(os.listdir(directory)): # Ordina per nome file
        if filename.lower().startswith("season") and filename.lower().endswith(".csv"):
            files_found += 1
            filepath = os.path.join(directory, filename)
            print(f"\nProcessando file: {filename}...")
            try:
                # Tenta diverse codifiche comuni
                try:
                    df = pd.read_csv(filepath, encoding='ISO-8859-1', low_memory=False)
                except UnicodeDecodeError:
                    print(f"  INFO: Decodifica ISO-8859-1 fallita, tentativo con UTF-8.")
                    try:
                         df = pd.read_csv(filepath, encoding='utf-8', low_memory=False)
                    except UnicodeDecodeError:
                         print(f"  ERRORE: Anche decodifica UTF-8 fallita. Impossibile leggere il file. Ignorato.")
                         continue # Salta al prossimo file

                files_processed += 1

                # --- Estrazione Robusta Stagione ---
                season_part = filename.lower().split('.')[0].replace("season", "").strip('_-')
                season_str = season_part # Default
                if len(season_part) == 4 and season_part.isdigit():
                     s_start = int(season_part[:2])
                     s_end = int(season_part[2:])
                     # Gestione cambio secolo (es. 9900 -> 99-00)
                     if s_start > s_end and s_start > 80 : # Assume anni > 80 siano 19xx
                         season_str = f"{s_start}-{s_end:02d}"
                     elif s_end == s_start + 1: # Formato standard es. 1920 -> 19-20
                         season_str = f"{s_start:02d}-{s_end:02d}"
                     else: # Mantieni formato se non chiaro
                         season_str = season_part
                elif len(season_part) == 2 and season_part.isdigit(): # caso es. 99 -> 99-00 (assunzione)
                     s_start = int(season_part)
                     s_end = s_start + 1
                     if s_end == 100: s_end_str = "00"
                     else: s_end_str = f"{s_end:02d}"
                     season_str = f"{s_start:02d}-{s_end_str}"
                elif '-' in season_part or '_' in season_part: # Formato già YY-YY o YY_YY
                     parts = season_part.replace('_','-').split('-')
                     if len(parts) == 2 and parts[0].isdigit() and parts[1].isdigit():
                          season_str = f"{parts[0]}-{parts[1]}" # Normalizza trattino

                df[COL_SEASON] = season_str
                print(f"  Stagione estratta: '{season_str}'")

                # --- Controllo Colonne e Tipi Base ---
                missing_cols = [col for col in required_core_columns if col not in df.columns]
                if missing_cols:
                    print(f"  ATTENZIONE: Mancano colonne fondamentali: {', '.join(missing_cols)}. File ignorato.")
                    files_processed -= 1 # Non contarlo come processato
                    continue

                # Converti gol a numerico subito, gestendo errori. Metti NaN se non convertibile.
                for col in [COL_FTHG, COL_FTAG]:
                     if col in df.columns:
                         original_dtype = df[col].dtype
                         df[col] = pd.to_numeric(df[col], errors='coerce')
                         # Stampa warning solo se c'erano valori non numerici convertiti a NaN
                         if df[col].isnull().any() and not original_dtype.kind in 'if': # Era stringa/object?
                             print(f"    ATTENZIONE: Colonna '{col}' conteneva valori non numerici, convertiti in NaN.")
                     else:
                         # Questo non dovrebbe accadere per le core columns, ma per sicurezza
                          print(f"    ERRORE INTERNO: Colonna core '{col}' inaspettatamente mancante.")
                          df[col] = np.nan # Aggiungi colonna NaN

                # Gestione Colonna Data (fondamentale per grafici temporali)
                if COL_DATE in df.columns:
                    original_non_null_dates = df[COL_DATE].notna().sum()
                    # Prova a inferire il formato, prova dayfirst=True
                    df[COL_DATE] = pd.to_datetime(df[COL_DATE], errors='coerce', dayfirst=True, infer_datetime_format=True)
                    dates_converted = df[COL_DATE].notna().sum()
                    if dates_converted < original_non_null_dates:
                         print(f"    ATTENZIONE: Non tutte le date sono state convertite correttamente ({dates_converted}/{original_non_null_dates} riuscite).")
                    df[COL_YEAR] = df[COL_DATE].dt.year # Estrai anno per aggregazioni
                else:
                    print(f"  ATTENZIONE: Colonna '{COL_DATE}' non trovata. Grafici basati sul tempo non saranno disponibili.")
                    df[COL_DATE] = pd.NaT # Aggiungi colonna NaT
                    df[COL_YEAR] = np.nan # Aggiungi colonna NaN

                # Seleziona solo colonne potenzialmente utili per ridurre memoria
                potential_cols = [COL_SEASON, COL_DATE, COL_YEAR, COL_HT, COL_AT, COL_FTHG, COL_FTAG, COL_FTR]
                cols_to_keep = [col for col in potential_cols if col in df.columns]
                all_data.append(df[cols_to_keep])

            except pd.errors.EmptyDataError:
                print(f"  ATTENZIONE: File vuoto. Ignorato.")
            except pd.errors.ParserError as e:
                print(f"  ERRORE di parsing: {e}. File ignorato.")
            except FileNotFoundError:
                 print(f"  ERRORE: File non trovato (imprevisto).") # Già gestito da os.listdir
            except Exception as e:
                print(f"  ERRORE GENERICO durante lettura/elaborazione base: {e}. File ignorato.")
                print(traceback.format_exc()) # Stampa traceback per debug

    print(f"\nFine caricamento. Trovati {files_found} file 'season*.csv'. Processati con successo: {files_processed}.")

    if not all_data:
        print("ERRORE CRITICO: Nessun dato valido caricato. Controllare i file CSV e la directory.")
        return pd.DataFrame()

    # Combina tutti i DataFrame caricati con successo
    combined_df = pd.concat(all_data, ignore_index=True)
    print(f"Dati combinati grezzi: {combined_df.shape[0]} righe.")

    # --- Pulizia Finale e Ordinamento ---
    # Riempi NaN nei gol con 0 e converti a intero
    for col in [COL_FTHG, COL_FTAG]:
         if col in combined_df.columns:
             combined_df[col] = combined_df[col].fillna(0).astype(int)
         else: # Crea colonna 0 se mancava (improbabile per core cols)
              combined_df[col] = 0


    # Ordinamento robusto Stagioni (formato 'YY-YY')
    try:
        combined_df[COL_SEASON] = combined_df[COL_SEASON].astype(str)
        # Estrai anno inizio (es. '19-20' -> 19). Gestisce errore se formato non è XX-YY
        def get_start_year(season_str):
            parts = season_str.split('-')
            if len(parts) == 2 and parts[0].isdigit():
                year = int(parts[0])
                # Gestione anni < 50 -> 20xx, anni >= 50 -> 19xx (euristica comune)
                return 2000 + year if year < 50 else 1900 + year
            return np.nan # Ritorna NaN se formato non riconosciuto
        combined_df['_sort_key'] = combined_df[COL_SEASON].apply(get_start_year)
        # Ordina prima per anno numerico, poi per stringa stagione (gestisce NaN)
        combined_df = combined_df.sort_values(by=['_sort_key', COL_SEASON], na_position='first').drop(columns=['_sort_key'])
        print("Ordinamento per stagione applicato.")
    except Exception as e:
        print(f"ATTENZIONE: Ordinamento complesso per stagione fallito ({e}). Fallback a ordinamento alfabetico.")
        combined_df = combined_df.sort_values(by=COL_SEASON)

    # Rimuovi duplicati (se Date esiste e valida) e righe con squadre mancanti
    initial_rows = len(combined_df)
    combined_df.dropna(subset=[COL_HT, COL_AT], inplace=True) # Richiede nomi squadre
    if COL_DATE in combined_df.columns and combined_df[COL_DATE].notna().any():
        combined_df.dropna(subset=[COL_DATE], inplace=True) # Rimuovi se data è NaN
        # Ordina per data prima di rimuovere duplicati per mantenere la prima occorrenza
        combined_df = combined_df.sort_values(by=COL_DATE)
        combined_df.drop_duplicates(subset=[COL_DATE, COL_HT, COL_AT], keep='first', inplace=True)
        print(f"Rimosse {initial_rows - len(combined_df)} righe duplicate o con data/squadre mancanti.")
    elif initial_rows > len(combined_df): # Se sono state rimosse solo per HT/AT mancanti
         print(f"Rimosse {initial_rows - len(combined_df)} righe con squadre mancanti.")


    print(f"Dati caricati e puliti: {combined_df.shape[0]} righe.")
    return combined_df

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    """Pulisce i dati e calcola colonne derivate (Gol Totali, Punti, etc.)."""
    if df.empty:
        return df
    print("\nInizio preprocessing colonne derivate...")

    # Assicura che FTHG/FTAG siano numerici interi (dovrebbe essere già così da load_data)
    for col in [COL_FTHG, COL_FTAG]:
        if col not in df.columns: df[col] = 0 # Aggiungi se mancante
        df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

    # Calcola colonne derivate Gol
    df[COL_TOTAL_GOALS] = df[COL_FTHG] + df[COL_FTAG]
    df[COL_GOAL_DIFF] = df[COL_FTHG] - df[COL_FTAG]
    df[COL_SCORE] = df[COL_FTHG].astype(str) + '-' + df[COL_FTAG].astype(str)
    print("  Calcolate colonne: TotalGoals, GoalDifference, Score.")

    # Calcola Punti
    if COL_FTR in df.columns:
        conditions_h = [df[COL_FTR] == 'H', df[COL_FTR] == 'D']; choices_h = [3, 1]
        conditions_a = [df[COL_FTR] == 'A', df[COL_FTR] == 'D']; choices_a = [3, 1]
        df[COL_PTS_H] = np.select(conditions_h, choices_h, default=0)
        df[COL_PTS_A] = np.select(conditions_a, choices_a, default=0)
        print("  Calcolate colonne: PointsH, PointsA.")
    else:
        print(f"  ATTENZIONE: Colonna '{COL_FTR}' non trovata. Punti (PointsH/PointsA) impostati a 0.")
        df[COL_PTS_H] = 0
        df[COL_PTS_A] = 0

    # Pulisci nomi squadre da spazi extra (importante per filtri/aggregazioni)
    if COL_HT in df.columns: df[COL_HT] = df[COL_HT].astype(str).str.strip()
    if COL_AT in df.columns: df[COL_AT] = df[COL_AT].astype(str).str.strip()

    print(f"Preprocessing completato. Shape finale dati: {df.shape}")
    # print(f"Colonne finali disponibili: {df.columns.tolist()}")
    return df

def configure_layout(fig: go.Figure, title: str, xaxis_title: Optional[str] = None, yaxis_title: Optional[str] = None, legend_title: Optional[str] = None) -> go.Figure:
    """Applica configurazione layout standard, migliorata e più flessibile."""
    fig.update_layout(
        title=f'<b>{title}</b>', title_x=0.5, title_font_size=20,
        font=dict(family="Arial, sans-serif", size=12, color='#e0e0e0'),
        paper_bgcolor='rgba(17,17,17,1)', plot_bgcolor='rgba(30,30,30,1)',
        template="plotly_dark",
        xaxis_title=f"<i>{xaxis_title}</i>" if xaxis_title else "",
        yaxis_title=f"<i>{yaxis_title}</i>" if yaxis_title else "",
        xaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgba(70,70,70,0.5)', linecolor='rgb(150,150,150)', ticks='outside', tickfont_color='rgb(150,150,150)', zeroline=False),
        yaxis=dict(showgrid=True, gridwidth=1, gridcolor='rgba(70,70,70,0.5)', linecolor='rgb(150,150,150)', ticks='outside', tickfont_color='rgb(150,150,150)', zeroline=False),
        margin=dict(l=70, r=40, t=90, b=70),
        legend=dict(
            title=f'<i>{legend_title}</i>' if legend_title else "",
            orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1,
            bgcolor='rgba(40,40,40,0.85)', bordercolor='grey', borderwidth=1,
            font=dict(size=11)
        ),
        hovermode='x unified',
        hoverlabel=dict(bgcolor="rgba(10,10,10,0.8)", font_size=13, font_family="Arial, sans-serif", bordercolor="rgba(200, 200, 200, 0.5)")
    )
    return fig

# --- Funzioni Grafici (Migliorate + Nuove) ---

# (Funzioni plot_goals_per_season, plot_home_away_goals_per_season, plot_goals_by_team,
#  plot_draw_percentage, plot_avg_goals_per_match, plot_goals_distribution_scatter,
#  plot_results_distribution_pie, plot_results_distribution_season,
#  plot_goal_difference_histogram, plot_most_frequent_scorelines, plot_points_per_season
#  vanno copiate/incollate qui dall'ultima versione, assicurandosi che siano aggiornate)
# ... (Ometto per brevità, sono le stesse 11 funzioni dell'ultima versione del codice) ...
# Copia qui le 11 funzioni di plot dall'ultima versione
def plot_goals_per_season(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_SEASON not in df or COL_TOTAL_GOALS not in df: return go.Figure().update_layout(title="Dati insufficienti per Gol Totali / Stagione")
    goals_per_season = df.groupby(COL_SEASON, observed=False)[COL_TOTAL_GOALS].sum().reset_index()
    fig = px.line(goals_per_season, x=COL_SEASON, y=COL_TOTAL_GOALS, markers=True, labels={COL_TOTAL_GOALS: 'Gol Totali', COL_SEASON: 'Stagione'})
    fig = configure_layout(fig, "Andamento Gol Totali per Stagione", "Stagione", "Numero di Gol")
    fig.update_traces(name="Gol Totali", showlegend=True, line=dict(color='#1f77b4', width=3), marker=dict(size=8, symbol='circle', line=dict(width=1, color='white')), hovertemplate='<b>Stagione:</b> %{x}<br><b>Gol Totali:</b> %{y:,}<extra></extra>')
    return fig
def plot_home_away_goals_per_season(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_SEASON not in df or COL_FTHG not in df or COL_FTAG not in df: return go.Figure().update_layout(title="Dati insufficienti per Media Gol Casa/Trasferta")
    matches_per_season = df.groupby(COL_SEASON, observed=False).size(); matches_per_season = matches_per_season[matches_per_season > 0]
    if matches_per_season.empty: return go.Figure().update_layout(title="Nessuna stagione con partite valide trovata.")
    goals_agg = df.groupby(COL_SEASON, observed=False)[[COL_FTHG, COL_FTAG]].sum(); goals_agg = goals_agg.reindex(matches_per_season.index)
    avg_goals = goals_agg.div(matches_per_season, axis=0).reset_index(); fig = go.Figure()
    fig.add_trace(go.Scatter(x=avg_goals[COL_SEASON], y=avg_goals[COL_FTHG], mode='lines+markers', name="Media Gol Squadra Casa", line=dict(color='#ff7f0e', width=3), marker=dict(size=8, symbol='square', line=dict(width=1, color='white')), hovertemplate='<b>Stagione:</b> %{x}<br><b>Media Gol Casa:</b> %{y:.2f}<extra></extra>'))
    fig.add_trace(go.Scatter(x=avg_goals[COL_SEASON], y=avg_goals[COL_FTAG], mode='lines+markers', name="Media Gol Squadra Trasferta", line=dict(color='#2ca02c', width=3), marker=dict(size=8, symbol='diamond', line=dict(width=1, color='white')), hovertemplate='<b>Stagione:</b> %{x}<br><b>Media Gol Trasferta:</b> %{y:.2f}<extra></extra>'))
    fig = configure_layout(fig, "Media Gol Casa vs Trasferta per Stagione", "Stagione", "Media Gol per Partita", legend_title="Squadra")
    return fig
def plot_goals_by_team(df: pd.DataFrame, team: str) -> go.Figure:
    if df.empty or not all(c in df.columns for c in [COL_SEASON, COL_HT, COL_AT, COL_FTHG, COL_FTAG]): return go.Figure().update_layout(title=f"Dati insufficienti per generare grafico per {team}")
    team_list = pd.concat([df[COL_HT], df[COL_AT]]).dropna().unique(); team_stripped = team.strip(); similar_teams = [t for t in team_list if t.strip().lower() == team_stripped.lower()]
    if similar_teams: team = similar_teams[0]; print(f"INFO: Trovata corrispondenza: '{team}'")
    elif team not in team_list: return go.Figure().update_layout(title=f"Squadra '{team}' non trovata.")
    home_goals = df[df[COL_HT] == team].groupby(COL_SEASON, observed=False)[COL_FTHG].sum(); away_goals = df[df[COL_AT] == team].groupby(COL_SEASON, observed=False)[COL_FTAG].sum()
    total_goals_team = home_goals.add(away_goals, fill_value=0).reset_index(); total_goals_team.columns = [COL_SEASON, 'TotalGoalsTeam']
    if total_goals_team.empty: return go.Figure().update_layout(title=f"Nessun dato gol per '{team}'.")
    fig = px.line(total_goals_team, x=COL_SEASON, y='TotalGoalsTeam', markers=True, labels={'TotalGoalsTeam': 'Gol Segnati', COL_SEASON: 'Stagione'})
    fig = configure_layout(fig, f'Andamento Gol Segnati: {team}', "Stagione", "Gol Segnati")
    fig.update_traces(name=f"Gol {team}", showlegend=True, line=dict(color='#d62728', width=3), marker=dict(size=8, symbol='cross', line=dict(width=1, color='white')), hovertemplate='<b>Stagione:</b> %{x}<br><b>Gol Segnati:</b> %{y}<extra></extra>')
    return fig
def plot_draw_percentage(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_SEASON not in df or COL_FTR not in df: return go.Figure().update_layout(title="Dati insufficienti per Percentuale Pareggi")
    total_matches = df.groupby(COL_SEASON, observed=False).size(); total_matches = total_matches[total_matches > 0]
    if total_matches.empty: return go.Figure().update_layout(title="Nessuna stagione con partite valide.")
    draws = df[df[COL_FTR] == 'D'].groupby(COL_SEASON, observed=False).size()
    draw_pct = (draws.reindex(total_matches.index, fill_value=0) / total_matches * 100).reset_index(); draw_pct.columns = [COL_SEASON, 'DrawPercentage']
    fig = px.line(draw_pct, x=COL_SEASON, y='DrawPercentage', markers=True, labels={'DrawPercentage': 'Percentuale Pareggi (%)', COL_SEASON: 'Stagione'})
    fig = configure_layout(fig, "Percentuale Pareggi per Stagione", "Stagione", "Percentuale (%)")
    fig.update_traces(name="Pareggi (%)", showlegend=True, line=dict(color='#9467bd', width=3), marker=dict(size=8, symbol='hourglass', line=dict(width=1, color='white')), hovertemplate='<b>Stagione:</b> %{x}<br><b>Pareggi:</b> %{y:.1f}%<extra></extra>')
    avg_draw_pct = draw_pct['DrawPercentage'].mean(); fig.add_hline(y=avg_draw_pct, line_width=2, line_dash="dash", line_color="grey", annotation_text=f"Media: {avg_draw_pct:.1f}%", annotation_position="bottom right")
    return fig
def plot_avg_goals_per_match(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_SEASON not in df or COL_TOTAL_GOALS not in df: return go.Figure().update_layout(title="Dati insufficienti per Media Gol/Partita")
    matches_per_season = df.groupby(COL_SEASON, observed=False).size(); matches_per_season = matches_per_season[matches_per_season > 0]
    if matches_per_season.empty: return go.Figure().update_layout(title="Nessuna stagione con partite valide.")
    total_goals_per_season = df.groupby(COL_SEASON, observed=False)[COL_TOTAL_GOALS].sum()
    avg_goals = (total_goals_per_season.reindex(matches_per_season.index) / matches_per_season).reset_index(); avg_goals.columns = [COL_SEASON, 'AverageGoals']
    fig = px.bar(avg_goals, x=COL_SEASON, y='AverageGoals', labels={'AverageGoals': 'Media Gol per Partita', COL_SEASON: 'Stagione'}, color='AverageGoals', color_continuous_scale=px.colors.sequential.YlGnBu, text='AverageGoals')
    fig.update_traces(texttemplate='%{text:.2f}', textposition='outside', marker_line_width=1.5, marker_line_color='white', hovertemplate='<b>Stagione:</b> %{x}<br><b>Media Gol:</b> %{y:.2f}<extra></extra>')
    fig = configure_layout(fig, "Media Gol Totali per Partita per Stagione", "Stagione", "Media Gol / Partita")
    fig.update_layout(bargap=0.2, coloraxis_colorbar=dict(title="Media Gol"))
    return fig
def plot_goals_distribution_scatter(df: pd.DataFrame) -> go.Figure:
    required = [COL_SEASON, COL_HT, COL_AT, COL_FTHG, COL_FTAG]
    if df.empty or not all(c in df.columns for c in required): return go.Figure().update_layout(title="Dati insufficienti per Scatter Gol Squadra")
    home_perf = df.groupby([COL_SEASON, COL_HT], observed=False).agg(GoalsScoredHome=(COL_FTHG, 'sum'), GoalsConcededHome=(COL_FTAG, 'sum')).reset_index().rename(columns={COL_HT: 'Team'})
    away_perf = df.groupby([COL_SEASON, COL_AT], observed=False).agg(GoalsScoredAway=(COL_FTAG, 'sum'), GoalsConcededAway=(COL_FTHG, 'sum')).reset_index().rename(columns={COL_AT: 'Team'})
    team_perf = pd.merge(home_perf, away_perf, on=[COL_SEASON, 'Team'], how='outer').fillna(0)
    team_perf['TotalGoalsScored'] = team_perf['GoalsScoredHome'] + team_perf['GoalsScoredAway']; team_perf['TotalGoalsConceded'] = team_perf['GoalsConcededHome'] + team_perf['GoalsConcededAway']
    num_cols = ['GoalsScoredHome', 'GoalsConcededHome', 'GoalsScoredAway', 'GoalsConcededAway', 'TotalGoalsScored', 'TotalGoalsConceded']
    for col in num_cols: team_perf[col] = team_perf[col].astype(int)
    if team_perf.empty: return go.Figure().update_layout(title="Nessun dato aggregato per squadre disponibile.")
    fig = px.scatter(team_perf, x='GoalsScoredHome', y='GoalsScoredAway', color=COL_SEASON, size='TotalGoalsScored', hover_name='Team', hover_data={COL_SEASON: True, 'GoalsScoredHome': ':.0f', 'GoalsScoredAway': ':.0f', 'TotalGoalsScored': ':.0f', 'TotalGoalsConceded': ':.0f', 'Team': False}, opacity=0.7, size_max=45, color_continuous_scale=px.colors.sequential.Viridis)
    fig = configure_layout(fig, "Distribuzione Gol Segnati Casa vs Trasferta per Squadra/Stagione", "Gol Segnati in Casa", "Gol Segnati in Trasferta", legend_title="Stagione")
    fig.update_layout(hovermode='closest', coloraxis_colorbar=dict(title="Stagione"))
    max_val = max(team_perf['GoalsScoredHome'].max(), team_perf['GoalsScoredAway'].max()) * 1.05; fig.add_shape(type="line", x0=0, y0=0, x1=max_val, y1=max_val, line=dict(color="grey", width=2, dash="dash"))
    fig.update_xaxes(range=[0, max_val]); fig.update_yaxes(range=[0, max_val])
    return fig
def plot_results_distribution_pie(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_FTR not in df: return go.Figure().update_layout(title="Dati insufficienti per Torta Risultati")
    results_counts = df[COL_FTR].value_counts(); labels_map = {'H': 'Vittorie Casa', 'D': 'Pareggi', 'A': 'Vittorie Trasferta'}; mapped_labels = [labels_map.get(i, i) for i in results_counts.index]
    colors_map = {'Vittorie Casa': '#1f77b4', 'Pareggi': '#7f7f7f', 'Vittorie Trasferta': '#2ca02c'}; mapped_colors = [colors_map.get(label, '#cccccc') for label in mapped_labels]
    fig = go.Figure(data=[go.Pie(labels=mapped_labels, values=results_counts.values, hole=0.4, marker_colors=mapped_colors, pull=[0.03 if l == 'Vittorie Casa' else 0 for l in mapped_labels])])
    fig = configure_layout(fig, "Distribuzione Risultati Complessiva (Tutte le Stagioni)", legend_title="Esito Partita")
    fig.update_traces(textposition='inside', textinfo='percent+label', insidetextorientation='radial', hoverinfo='label+percent+value', marker=dict(line=dict(color='#FFFFFF', width=2.5)))
    fig.update_layout(showlegend=True)
    return fig
def plot_results_distribution_season(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_SEASON not in df or COL_FTR not in df: return go.Figure().update_layout(title="Dati insufficienti per Barre Risultati/Stagione")
    results_by_season = df.groupby(COL_SEASON, observed=False)[COL_FTR].value_counts(normalize=True).unstack(fill_value=0) * 100
    labels_map = {'H': 'Vittorie Casa', 'D': 'Pareggi', 'A': 'Vittorie Trasferta'}; colors_map = {'Vittorie Casa': '#1f77b4', 'Pareggi': '#7f7f7f', 'Vittorie Trasferta': '#2ca02c'}
    results_by_season.rename(columns=labels_map, inplace=True); ordered_cols = [labels_map[k] for k in ['H', 'D', 'A']]
    for col in ordered_cols:
        if col not in results_by_season.columns: results_by_season[col] = 0
    results_by_season = results_by_season[ordered_cols]
    if results_by_season.empty: return go.Figure().update_layout(title="Nessun dato aggregato per risultati/stagione disponibile.")
    fig = px.bar(results_by_season, x=results_by_season.index, y=ordered_cols, labels={'value': 'Percentuale (%)', COL_SEASON: 'Stagione', 'variable': 'Esito Partita'}, color_discrete_map=colors_map)
    fig = configure_layout(fig, "Distribuzione Percentuale Risultati per Stagione", "Stagione", "Percentuale (%)", legend_title="Esito Partita")
    fig.update_layout(barmode='stack', yaxis_ticksuffix="%")
    fig.update_traces(hovertemplate='<b>Stagione:</b> %{x}<br><b>Esito:</b> %{data.name}<br><b>Percentuale:</b> %{y:.1f}%<extra></extra>')
    return fig
def plot_goal_difference_histogram(df: pd.DataFrame) -> go.Figure:
    if df.empty or COL_GOAL_DIFF not in df: return go.Figure().update_layout(title="Dati insufficienti per Istogramma Diff. Reti")
    min_diff = df[COL_GOAL_DIFF].min(); max_diff = df[COL_GOAL_DIFF].max(); nbins = max(15, min(50, max_diff - min_diff + 1))
    fig = px.histogram(df, x=COL_GOAL_DIFF, nbins=nbins, labels={COL_GOAL_DIFF: 'Differenza Reti (Gol Casa - Gol Trasferta)', 'count': 'Numero Partite'}, opacity=0.8, color_discrete_sequence=['#636EFA'])
    fig = configure_layout(fig, "Distribuzione Differenza Reti (Gol Casa - Gol Trasferta)", "Differenza Reti", "Numero di Partite")
    fig.update_layout(bargap=0.1, hovermode='x')
    fig.add_vline(x=0, line_width=2, line_dash="dash", line_color="red", annotation_text="Pareggio (0)", annotation_position="top right")
    fig.update_traces(hovertemplate='<b>Diff. Reti:</b> %{x}<br><b>N. Partite:</b> %{y}<extra></extra>', marker_line_width=0.5, marker_line_color='white')
    fig.update_layout(showlegend=False)
    return fig
def plot_most_frequent_scorelines(df: pd.DataFrame, top_n: int = 15) -> go.Figure:
    if df.empty or COL_SCORE not in df: return go.Figure().update_layout(title="Dati insufficienti per Punteggi Frequenti")
    score_counts = df[COL_SCORE].value_counts().nlargest(top_n).reset_index(); score_counts.columns = [COL_SCORE, 'Frequency']; score_counts[COL_SCORE] = score_counts[COL_SCORE].astype(str)
    fig = px.bar(score_counts, y=COL_SCORE, x='Frequency', orientation='h', labels={COL_SCORE: 'Punteggio Finale', 'Frequency': 'Numero di Partite'}, color='Frequency', color_continuous_scale=px.colors.sequential.Plasma_r, text='Frequency')
    fig = configure_layout(fig, f"Top {top_n} Punteggi più Frequenti", "Numero di Partite", "Punteggio")
    fig.update_yaxes(categoryorder='total ascending'); fig.update_layout(hovermode='y unified', coloraxis_colorbar=dict(title="Frequenza"))
    fig.update_traces(hovertemplate='<b>Punteggio:</b> %{y}<br><b>N. Partite:</b> %{x}<extra></extra>', textposition='outside', marker_line_color='white', marker_line_width=1)
    fig.update_layout(showlegend=False)
    return fig
def plot_points_per_season(df: pd.DataFrame, team: Optional[str] = None) -> go.Figure:
    required_cols = [COL_SEASON, COL_PTS_H, COL_PTS_A, COL_HT, COL_AT]
    if df.empty or not all(col in df.columns for col in required_cols): return go.Figure().update_layout(title="Dati insufficienti per Andamento Punti")
    if team:
        team_list = pd.concat([df[COL_HT], df[COL_AT]]).dropna().unique(); team_stripped = team.strip(); similar_teams = [t for t in team_list if t.strip().lower() == team_stripped.lower()]
        if similar_teams: team = similar_teams[0]; print(f"INFO: Trovata corrispondenza per punti: '{team}'")
        elif team not in team_list: return go.Figure().update_layout(title=f"Squadra '{team}' non trovata per punti.")
        points_home = df[df[COL_HT] == team].groupby(COL_SEASON, observed=False)[COL_PTS_H].sum(); points_away = df[df[COL_AT] == team].groupby(COL_SEASON, observed=False)[COL_PTS_A].sum()
        total_points = points_home.add(points_away, fill_value=0).reset_index(); total_points.columns = [COL_SEASON, 'TotalPoints']
        if total_points.empty: return go.Figure().update_layout(title=f"Nessun dato punti per '{team}'.")
        plot_title = f"Andamento Punti Stagionali: {team}"; y_col, y_label, hover_label, trace_name = 'TotalPoints', "Punti Totali", "Punti", f"Punti {team}"; line_color = '#E377C2'; marker_symbol = 'star'
    else:
        total_points = df.groupby(COL_SEASON, observed=False)[[COL_PTS_H, COL_PTS_A]].sum().sum(axis=1).reset_index(); total_points.columns = [COL_SEASON, 'TotalLeaguePoints']
        if total_points.empty: return go.Figure().update_layout(title="Nessun dato punti campionato.")
        plot_title = "Andamento Punti Totali nel Campionato per Stagione"; y_col, y_label, hover_label, trace_name = 'TotalLeaguePoints', "Punti Totali Campionato", "Punti Camp.", "Punti Campionato"; line_color = '#BCBD22'; marker_symbol = 'triangle-up'
    fig = px.line(total_points, x=COL_SEASON, y=y_col, markers=True, labels={y_col: y_label, COL_SEASON: 'Stagione'})
    fig = configure_layout(fig, plot_title, "Stagione", y_label, legend_title="Dato")
    fig.update_traces(name=trace_name, showlegend=True, line=dict(color=line_color, width=3), marker=dict(size=9, symbol=marker_symbol, line=dict(width=1, color='white')), hovertemplate=f'<b>Stagione:</b> %{{x}}<br><b>{hover_label}:</b> %{{y}}<extra></extra>')
    return fig

# --- NUOVI GRAFICI ---

def plot_rolling_avg_goals(df: pd.DataFrame, window: int = 10) -> go.Figure:
    """Grafico a linee: media mobile dei gol totali per partita nel tempo."""
    required = [COL_DATE, COL_TOTAL_GOALS]
    if df.empty or not all(c in df.columns for c in required) or df[COL_DATE].isnull().all():
        return go.Figure().update_layout(title="Dati/Date insufficienti per Media Mobile Gol")

    # Assicura che i dati siano ordinati per data
    df_sorted = df.dropna(subset=[COL_DATE]).sort_values(by=COL_DATE)
    if df_sorted.empty:
         return go.Figure().update_layout(title="Nessuna partita con data valida per Media Mobile Gol")

    # Calcola la media mobile dei gol totali
    df_sorted['RollingAvgGoals'] = df_sorted[COL_TOTAL_GOALS].rolling(window=window, min_periods=max(1, window // 2)).mean() # min_periods per avere valori all'inizio

    fig = px.line(df_sorted, x=COL_DATE, y='RollingAvgGoals',
                  labels={'RollingAvgGoals': f'Media Mobile Gol ({window} part.)', COL_DATE: 'Data'})
    fig = configure_layout(fig, f"Andamento Media Mobile Gol Totali per Partita (Finestra: {window} partite)",
                           "Data", f"Media Mobile Gol ({window} part.)")
    fig.update_traces(
        name=f"Media Mobile ({window}p)", showlegend=True,
        line=dict(color='#8c564b', width=2.5), # Marrone
        hovertemplate='<b>Data:</b> %{x|%d %b %Y}<br><b>Media Mobile Gol:</b> %{y:.2f}<extra></extra>' # Formato data hover
    )
    # Rimuovi markers per rendere linea più pulita
    fig.update_traces(mode='lines')
    return fig

def plot_goal_diff_team_season(df: pd.DataFrame) -> go.Figure:
    """Grafico a barre: differenza reti totale per squadra per stagione."""
    required = [COL_SEASON, COL_HT, COL_AT, COL_FTHG, COL_FTAG]
    if df.empty or not all(c in df.columns for c in required):
        return go.Figure().update_layout(title="Dati insufficienti per Diff. Reti Squadra/Stagione")

    # Calcola gol fatti/subiti in casa
    home_perf = df.groupby([COL_SEASON, COL_HT], observed=False).agg(
        GF_H=(COL_FTHG, 'sum'), GS_H=(COL_FTAG, 'sum')
    ).reset_index().rename(columns={COL_HT: 'Team'})
    # Calcola gol fatti/subiti in trasferta
    away_perf = df.groupby([COL_SEASON, COL_AT], observed=False).agg(
        GF_A=(COL_FTAG, 'sum'), GS_A=(COL_FTHG, 'sum')
    ).reset_index().rename(columns={COL_AT: 'Team'})

    # Unisce e calcola totali e differenza reti
    team_perf = pd.merge(home_perf, away_perf, on=[COL_SEASON, 'Team'], how='outer').fillna(0)
    team_perf['TotalGF'] = team_perf['GF_H'] + team_perf['GF_A']
    team_perf['TotalGS'] = team_perf['GS_H'] + team_perf['GS_A']
    team_perf['TotalGD'] = (team_perf['TotalGF'] - team_perf['TotalGS']).astype(int) # Differenza Reti

    # Rimuovi squadre con 0 partite (se merge outer ha creato righe solo Nan)
    team_perf = team_perf[(team_perf['TotalGF'] != 0) | (team_perf['TotalGS'] != 0)]

    if team_perf.empty:
         return go.Figure().update_layout(title="Nessun dato aggregato per squadre disponibile.")

    # Ordinamento per stagione e poi per diff reti decrescente
    team_perf = team_perf.sort_values(by=[COL_SEASON, 'TotalGD'], ascending=[True, False])

    fig = px.bar(team_perf, x='Team', y='TotalGD', color=COL_SEASON,
                 labels={'TotalGD': 'Differenza Reti Totale', 'Team': 'Squadra', COL_SEASON: 'Stagione'},
                 barmode='group', # Barre raggruppate per squadra, non impilate
                 facet_row=COL_SEASON, # Crea un subplot per ogni stagione
                 height=300 * team_perf[COL_SEASON].nunique() # Altezza dinamica basata sul numero di stagioni
                )

    fig = configure_layout(fig, "Differenza Reti Totale per Squadra e Stagione",
                           "Squadra", "Differenza Reti") # Titoli assi principali

    # Migliora layout subplot
    fig.update_yaxes(matches=None, showticklabels=True) # Mostra etichette Y per ogni subplot
    fig.update_xaxes(showticklabels=True, tickangle=45) # Mostra etichette X (squadre) inclinate
    fig.update_layout(bargap=0.3)

    # Aggiorna hovertemplate e aspetto barre
    fig.update_traces(hovertemplate='<b>Squadra:</b> %{x}<br><b>Stagione:</b> %{customdata[0]}<br><b>Diff. Reti:</b> %{y}<extra></extra>',
                      customdata=team_perf[[COL_SEASON]], # Aggiunge stagione ai dati hover
                      marker_line_width=0.5, marker_line_color='white')

    # Nasconde la legenda duplicata dei colori stagione (già indicata dai titoli subplot)
    fig.update_layout(showlegend=False)
    # Rimuovi titoli subplot generati automaticamente (facet_row)
    fig.for_each_annotation(lambda a: a.update(text=""))

    return fig


# --- Gestione Interfaccia ---

# Mappa nomi grafici a funzioni
PLOT_FUNCTIONS: Dict[str, Dict[str, Any]] = {
    "Gol Totali / Stagione": {"func": plot_goals_per_season, "params": None},
    "Media Gol Casa vs Trasferta / Stagione": {"func": plot_home_away_goals_per_season, "params": None},
    "Percentuale Pareggi / Stagione": {"func": plot_draw_percentage, "params": None},
    "Media Gol Totali / Partita / Stagione": {"func": plot_avg_goals_per_match, "params": None},
    "Distribuzione Risultati (Torta Generale)": {"func": plot_results_distribution_pie, "params": None},
    "Distribuzione Risultati / Stagione (%)": {"func": plot_results_distribution_season, "params": None},
    "Distribuzione Differenza Reti (Istogramma)": {"func": plot_goal_difference_histogram, "params": None},
    "Punteggi più Frequenti (Top 15)": {"func": plot_most_frequent_scorelines, "params": {'top_n': 15}}, # Passa parametro
    "Distribuzione Gol Squadra (Scatter Casa/Trasf)": {"func": plot_goals_distribution_scatter, "params": None},
    "Andamento Punti Totali Campionato": {"func": plot_points_per_season, "params": None},
    "Media Mobile Gol Totali (Finestra 10 part.)": {"func": plot_rolling_avg_goals, "params": {'window': 10}}, # Nuovo
    "Differenza Reti Squadra per Stagione": {"func": plot_goal_diff_team_season, "params": None}, # Nuovo
    "--- Grafici per Squadra Selezionata ---": {"func": None, "params": None}, # Separatore
    "Andamento Gol Squadra": {"func": plot_goals_by_team, "params": "team"},
    "Andamento Punti Squadra": {"func": plot_points_per_season, "params": "team"},
}

# --- Esecuzione Principale ---

# QUI DOVETE INSERIRE IL PERCORSO DELLA CARTELLA DOVE AVETE SCARICATO I FILE
directory = "/Users/giovannisassano/Desktop/MACHINE LEARNING POLIMI/Programming/Partite/Stagioni/" # <--- PERCORSO SPECIFICATO

# Carica e preprocessa i dati
raw_data = load_data(directory)
data = preprocess_data(raw_data.copy())

# --- Creazione widget interattivi ---
chart_options = [name for name, info in PLOT_FUNCTIONS.items() if info["func"] is not None or "---" in name] # Include separatore
chart_dropdown = widgets.Dropdown(options=chart_options, value=chart_options[0], description='Seleziona Grafico:', style={'description_width': 'initial'}, layout=Layout(width='95%'))

if not data.empty and COL_HT in data and COL_AT in data:
     team_options = sorted(list(pd.concat([data[COL_HT], data[COL_AT]]).astype(str).dropna().unique()))
     team_options = [""] + team_options
else: team_options = [""]
team_dropdown = widgets.Dropdown(options=team_options, value="", description='Seleziona Squadra:', style={'description_width': 'initial'}, layout=Layout(width='95%', visibility='hidden'), disabled=True)

output_area = widgets.Output()

# Funzione di aggiornamento grafico
def update_chart(change: Optional[Dict] = None) -> None:
    selected_chart_name = chart_dropdown.value
    if selected_chart_name not in PLOT_FUNCTIONS or PLOT_FUNCTIONS[selected_chart_name]["func"] is None:
        team_dropdown.layout.visibility = 'hidden'; team_dropdown.disabled = True
        with output_area: clear_output(wait=True); print("Seleziona un tipo di grafico valido dalla lista.")
        return

    plot_info = PLOT_FUNCTIONS[selected_chart_name]
    plot_func = plot_info["func"]
    params_info = plot_info["params"]

    requires_team = isinstance(params_info, str) and params_info == "team"
    if requires_team: team_dropdown.layout.visibility = 'visible'; team_dropdown.disabled = False
    else: team_dropdown.layout.visibility = 'hidden'; team_dropdown.disabled = True

    with output_area:
        clear_output(wait=True)
        if data.empty: print("ERRORE: Dati non caricati o vuoti."); return
        try:
            fig: Optional[go.Figure] = None
            kwargs = {} # Argomenti keyword da passare alla funzione plot

            # Gestione parametri specifici
            if isinstance(params_info, dict): # Es. {'window': 10} o {'top_n': 15}
                 kwargs.update(params_info)
            elif requires_team:
                selected_team = team_dropdown.value
                if not selected_team: print("INFO: Seleziona una squadra per questo grafico."); return
                kwargs['team'] = selected_team # Passa la squadra

            print(f"Generazione grafico: '{selected_chart_name}'...")
            fig = plot_func(data, **kwargs) # Chiama con dati e kwargs
            print("Grafico generato.")

            if fig:
                fig.show(renderer="vscode" if "VSCODE_PID" in os.environ else None)
                print("Grafico visualizzato.")
            else: print(f"ATTENZIONE: Funzione per '{selected_chart_name}' non ha restituito figura.")

        except Exception as e:
            print(f"ERRORE durante generazione grafico '{selected_chart_name}':"); print(f"{type(e).__name__}: {e}")
            print("--- Traceback ---"); print(traceback.format_exc()); print("--- Fine Traceback ---")

# Osserva cambiamenti e organizza UI
chart_dropdown.observe(update_chart, names='value')
team_dropdown.observe(update_chart, names='value')
ui = VBox([chart_dropdown, team_dropdown, output_area], layout=Layout(width='95%', max_width='950px', padding="15px", border='1px solid #444', margin='10px auto'))

# Visualizza UI e grafico iniziale
print("\nInterfaccia utente pronta. Caricamento grafico iniziale...")
display(ui)
update_chart() # Mostra il primo grafico della lista
print("--- Script completato ---")

PREDIZIONE


In [ ]:
import os
import pandas as pd
import numpy as np
import random
from collections import defaultdict
from tabulate import tabulate
from IPython.display import display, HTML
import math
from tqdm.notebook import tqdm  # Per barra di avanzamento (opzionale, richiede tqdm e ipywidgets)

# --- 1. CONFIGURAZIONE ---
DATA_DIRECTORY = "/Users/giovannisassano/Desktop/MACHINE LEARNING POLIMI/Programming/Partite/Stagioni/"
# Iperparametri
LEARNING_RATE = 0.1  # Tasso di apprendimento (Alpha)
DISCOUNT_FACTOR = 0.9  # Fattore di sconto (Gamma)
EXPLORATION_PROB_START = 0.5  # Epsilon iniziale per esplorazione
EXPLORATION_PROB_END = 0.01  # Epsilon finale
EXPLORATION_DECAY_RATE = 0.0005  # Tasso di decadimento di epsilon
TRAINING_EPISODES = 5  # Numero di episodi di allenamento (passate sui dati)

N_RECENT_GAMES = 5  # Numero partite per calcolare la forma recente

# Spazio delle azioni (Risultati possibili)
POSSIBLE_ACTIONS = ['H', 'D', 'A']
NUM_ACTIONS = len(POSSIBLE_ACTIONS)

# Mappa azioni a simboli schedina
ACTION_TO_SYMBOL = {'H': '1', 'D': 'X', 'A': '2'}
# Mappa azioni a icone
ACTION_TO_ICON = {'H': '🏠', 'D': '🤝', 'A': '✈️'}

# Definisci i bin per la discretizzazione (ESEMPIO - DA OTTIMIZZARE!)
GOAL_BINS = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0]  # Media gol
POINTS_BINS = [0.5, 1.0, 1.5, 2.0, 2.5]  # Media punti
H2H_BINS = [0.25, 0.5, 0.75]  # Percentuale H2H


# --- 2. CARICAMENTO E PREPROCESSING DATI ---

def load_data(directory: str) -> pd.DataFrame:
    """
    Carica e combina i dati delle stagioni dai file CSV.

    Args:
        directory (str): Il percorso della directory contenente i file CSV.

    Returns:
        pd.DataFrame: Un DataFrame contenente tutti i dati combinati e pre-processati,
                      oppure un DataFrame vuoto in caso di errori.
    """
    all_data = []
    required_columns = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'Season']
    print(f"Caricamento dati da: {directory}")
    if not os.path.isdir(directory):
        print(f"Errore: Directory '{directory}' non trovata.")
        return pd.DataFrame()

    for filename in os.listdir(directory):
        if filename.startswith("season") and filename.endswith(".csv"):
            filepath = os.path.join(directory, filename)
            try:
                df = pd.read_csv(filepath, encoding='ISO-8859-1')

                # Estrai stagione dal nome file se non presente come colonna
                if 'Season' not in df.columns:
                    season_str = filename.split('.')[0].replace("season", "")
                    df['Season'] = season_str
                elif df['Season'].isnull().any():
                    print(f"Attenzione: Colonna 'Season' contiene valori mancanti nel file '{filename}'. Potrebbe essere necessario un processamento aggiuntivo.")

                # Controllo colonne essenziali
                if not all(col in df.columns for col in ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']):
                    print(f"Attenzione: File '{filename}' mancano colonne essenziali. Ignorato.")
                    continue

                # Assicura tipi di dati corretti e gestione degli errori
                df['FTHG'] = pd.to_numeric(df['FTHG'], errors='coerce').astype('Int64')  # Usa 'Int64' per permettere NaN
                df['FTAG'] = pd.to_numeric(df['FTAG'], errors='coerce').astype('Int64')
                df['Date'] = pd.to_datetime(df['Date'], errors='coerce', dayfirst=True)

                # Rimuovi righe con valori mancanti essenziali dopo la conversione
                df.dropna(subset=['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR'], inplace=True)

                # Assicura che FTHG/FTAG siano interi (dopo aver rimosso i NaN)
                df['FTHG'] = df['FTHG'].astype(int)
                df['FTAG'] = df['FTAG'].astype(int)

                # Prepara la colonna Season se manca (ora gestita all'inizio del loop)

                # Controlla se tutte le colonne richieste sono ora presenti
                if all(col in df.columns for col in required_columns):
                    all_data.append(df[required_columns].copy())  # Seleziona solo le colonne necessarie e crea una copia
                else:
                    missing = [col for col in required_columns if col not in df.columns]
                    print(f"Attenzione: File '{filename}' mancano ancora colonne dopo processamento: {missing}. Ignorato.")

            except Exception as e:
                print(f"Errore durante la lettura o processamento di '{filename}': {e}")

    if not all_data:
        print("Errore: Nessun dato caricato.")
        return pd.DataFrame()

    combined_data = pd.concat(all_data, ignore_index=True)
    # Ordina per data - FONDAMENTALE per calcolare la forma!
    combined_data = combined_data.sort_values(by='Date').reset_index(drop=True)
    print(f"Dati caricati e ordinati: {combined_data.shape[0]} partite.")
    return combined_data


# --- 3. FEATURE ENGINEERING ---

def calculate_rolling_stats(team, date, team_data, n_games=N_RECENT_GAMES):
    """Calcola statistiche medie mobili (gol fatti/subiti) per una squadra PRIMA di una certa data."""
    past_games = team_data[team_data['Date'] < date].tail(n_games)
    if past_games.empty:
        return 0.0, 0.0
    avg_goals_scored = past_games['GoalsScored'].mean()
    avg_goals_conceded = past_games['GoalsConceded'].mean()
    return avg_goals_scored, avg_goals_conceded


def calculate_form_points(team, date, team_data, n_games=N_RECENT_GAMES):
    """Calcola i punti medi ottenuti nelle ultime N partite PRIMA di una certa data."""
    past_games = team_data[team_data['Date'] < date].tail(n_games)
    if past_games.empty:
        return 0.0
    points = past_games['Result'].apply(lambda r: 3 if r == 'W' else 1 if r == 'D' else 0)
    return points.mean()


def calculate_h2h_stats(home_team, away_team, date, all_data):
    """Calcola statistiche testa a testa (percentuale vittorie H/D/A) PRIMA di una data."""
    h2h_games = all_data[
        ((all_data['HomeTeam'] == home_team) & (all_data['AwayTeam'] == away_team) |
         (all_data['HomeTeam'] == away_team) & (all_data['AwayTeam'] == home_team)) &
        (all_data['Date'] < date)
        ].copy()  # Usa .copy() per evitare SettingWithCopyWarning
    if h2h_games.empty:
        return 0.0, 0.0, 0.0

    home_wins = 0
    draws = 0
    away_wins = 0
    for _, row in h2h_games.iterrows():
        if row['FTR'] == 'H':
            if row['HomeTeam'] == home_team:
                home_wins += 1
            else:
                away_wins += 1
        elif row['FTR'] == 'D':
            draws += 1
        elif row['FTR'] == 'A':
            if row['AwayTeam'] == home_team:
                home_wins += 1
            else:
                away_wins += 1

    total_games = len(h2h_games)
    return home_wins / total_games, draws / total_games, away_wins / total_games


def add_team_perspective_data(data):
    """Aggiunge colonne ausiliarie per facilitare il calcolo delle statistiche per squadra."""
    home_view = data[['Date', 'HomeTeam', 'FTHG', 'FTAG', 'FTR']].copy()
    home_view.rename(columns={'HomeTeam': 'Team', 'FTHG': 'GoalsScored', 'FTAG': 'GoalsConceded'}, inplace=True)
    home_view['Result'] = home_view['FTR'].apply(lambda x: 'W' if x == 'H' else ('D' if x == 'D' else 'L'))
    home_view['Location'] = 'Home'

    away_view = data[['Date', 'AwayTeam', 'FTAG', 'FTHG', 'FTR']].copy()
    away_view.rename(columns={'AwayTeam': 'Team', 'FTAG': 'GoalsScored', 'FTHG': 'GoalsConceded'}, inplace=True)
    away_view['Result'] = away_view['FTR'].apply(lambda x: 'W' if x == 'A' else ('D' if x == 'D' else 'L'))
    away_view['Location'] = 'Away'

    team_data = pd.concat([home_view, away_view], ignore_index=True)
    team_data = team_data.sort_values(by=['Team', 'Date']).reset_index(drop=True)
    return team_data


# --- 4. DISCRETIZZAZIONE E CREAZIONE STATO ---

def discretize_value(value, bins):
    """Discretizza un valore usando i bin forniti."""
    return np.digitize(value, bins, right=True)


def create_discrete_state(home_team, away_team, date, all_data, team_perspective_data):
    """Crea uno stato discreto per la partita basato sulle feature calcolate."""
    home_team_data = team_perspective_data[team_perspective_data['Team'] == home_team]
    away_team_data = team_perspective_data[team_perspective_data['Team'] == away_team]

    # Calcola features (medie mobili PRIMA della partita)
    home_avg_gs, home_avg_gc = calculate_rolling_stats(home_team, date, home_team_data)
    away_avg_gs, away_avg_gc = calculate_rolling_stats(away_team, date, away_team_data)
    home_form_points = calculate_form_points(home_team, date, home_team_data)
    away_form_points = calculate_form_points(away_team, date, away_team_data)
    h2h_hw_pct, h2h_d_pct, h2h_aw_pct = calculate_h2h_stats(home_team, away_team, date, all_data)

    # Discretizza le features
    state = (
        discretize_value(home_avg_gs, GOAL_BINS),
        discretize_value(home_avg_gc, GOAL_BINS),
        discretize_value(away_avg_gs, GOAL_BINS),
        discretize_value(away_avg_gc, GOAL_BINS),
        discretize_value(home_form_points, POINTS_BINS),
        discretize_value(away_form_points, POINTS_BINS),
        discretize_value(h2h_hw_pct, H2H_BINS),  # Percentuale vittorie H per la squadra di casa attuale
        discretize_value(h2h_d_pct, H2H_BINS),
    )
    return state


# --- 5. AGENTE Q-LEARNING ---

q_table = defaultdict(lambda: {action: 0.0 for action in POSSIBLE_ACTIONS})


def choose_action_epsilon_greedy(state_tuple, episode, epsilon_start, epsilon_end, epsilon_decay_rate):
    """Sceglie un'azione usando la strategia epsilon-greedy."""
    epsilon = epsilon_end + (epsilon_start - epsilon_end) * np.exp(-epsilon_decay_rate * episode)
    if random.uniform(0, 1) < epsilon:
        return random.choice(POSSIBLE_ACTIONS)  # Esplora
    else:
        # Sfrutta: scegli l'azione migliore basata sulla Q-table attuale
        return max(q_table[state_tuple], key=q_table[state_tuple].get)


def train_q_learning(data, episodes, alpha, gamma, eps_start, eps_end, eps_decay):
    """Allena il modello Q-learning sui dati storici."""
    print(f"Inizio addestramento Q-Learning per {episodes} episodi...")
    team_perspective_data = add_team_perspective_data(data)

    for episode in tqdm(range(episodes), desc="Addestramento Episodi"):
        for index, row in data.iterrows():
            home_team = row['HomeTeam']
            away_team = row['AwayTeam']
            actual_result = row['FTR']
            match_date = row['Date']

            # Crea lo stato discreto per la partita corrente
            try:
                state_tuple = create_discrete_state(home_team, away_team, match_date, data, team_perspective_data)
            except Exception as e:
                print(f"Errore nella creazione dello stato per {home_team} vs {away_team} il {match_date}: {e}")
                continue  # Salta questa partita in caso di errore

            # Scegli un'azione
            action = choose_action_epsilon_greedy(state_tuple, episode, eps_start, eps_end, eps_decay)

            # Calcola la ricompensa
            reward = 1 if action == actual_result else -1

            # Aggiorna la Q-table
            current_q_value = q_table[state_tuple][action]
            new_q_value = current_q_value + alpha * (reward - current_q_value)
            q_table[state_tuple][action] = new_q_value

    print("Addestramento completato.")
    print(f"Numero di stati unici esplorati: {len(q_table)}")


# --- 6. PREDIZIONE ---

def predict_match_outcome(home_team, away_team, prediction_date, all_data, team_perspective_data, current_q_table):
    """Predice l'esito di una singola partita futura."""
    try:
        state_tuple = create_discrete_state(home_team, away_team, prediction_date, all_data, team_perspective_data)
    except Exception as e:
        print(f"Errore nella creazione dello stato per la previsione di {home_team} vs {away_team}: {e}")
        return random.choice(POSSIBLE_ACTIONS), {action: 0.0 for action in POSSIBLE_ACTIONS}

    if state_tuple in current_q_table:
        predicted_action = max(current_q_table[state_tuple], key=current_q_table[state_tuple].get)
        q_values = current_q_table[state_tuple]
    else:
        # print(f"Attenzione: Stato non visto per {home_team} vs {away_team}. Predizione casuale.")
        predicted_action = random.choice(POSSIBLE_ACTIONS)
        q_values = {action: 0.0 for action in POSSIBLE_ACTIONS}

    return predicted_action, q_values


def generate_schedina_predictions(matches_to_predict, historical_data, current_q_table):
    """Genera le previsioni per una lista di partite."""
    predictions = []
    last_historical_date = historical_data['Date'].max() + pd.Timedelta(days=1)
    team_perspective_data = add_team_perspective_data(historical_data)

    print(f"\nGenerazione previsioni per {len(matches_to_predict)} partite (basato su dati fino a {last_historical_date.date()}):")
    for home, away in tqdm(matches_to_predict, desc="Previsione Partite"):
        predicted_action, q_vals = predict_match_outcome(home, away, last_historical_date, historical_data,
                                                          team_perspective_data, current_q_table)
        predictions.append({
            "HomeTeam": home,
            "AwayTeam": away,
            "Prediction": ACTION_TO_SYMBOL.get(predicted_action, '?'),
            "Icon": ACTION_TO_ICON.get(predicted_action, '❓'),
            "Q_H": q_vals.get('H', 0.0),
            "Q_D": q_vals.get('D', 0.0),
            "Q_A": q_vals.get('A', 0.0),
            "Predicted_Action": predicted_action  # Mantiene l'azione originale H/D/A
        })
    return predictions


# --- 7. OUTPUT SCHEDINA ---

def display_schedina_html(prediction_results):
    """Crea e visualizza una tabella HTML per la schedina."""
    # Calcola Softmax per ottenere pseudo-probabilità dai Q-values
    for result in prediction_results:
        q_values_array = np.array([result['Q_H'], result['Q_D'], result['Q_A']])
        # Aggiungi stabilità numerica (sottrai max Q) ed evita exp(inf)
        stable_q = q_values_array - np.max(q_values_array)
        exp_q = np.exp(stable_q)
        softmax_probs = exp_q / np.sum(exp_q)
        result['Prob_1'] = softmax_probs[0]  # Probabilità per 'H'
        result['Prob_X'] = softmax_probs[1]  # Probabilità per 'D'
        result['Prob_2'] = softmax_probs[2]  # Probabilità per 'A'
        # Determina confidenza come probabilità della classe predetta
        if result['Predicted_Action'] == 'H':
            result['Confidence'] = result['Prob_1']
        elif result['Predicted_Action'] == 'D':
            result['Confidence'] = result['Prob_X']
        else:
            result['Confidence'] = result['Prob_2']

    # Costruzione tabella HTML
    html = """
    <style>
        .schedina-table {
            border-collapse: collapse;
            width: 95%;
            margin: 20px auto;
            font-family: Arial, sans-serif;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
        }
        .schedina-table th, .schedina-table td {
            border: 1px solid #ddd;
            padding: 10px 12px;
            text-align: center;
        }
        .schedina-table th {
            background-color: #4CAF50; /* Verde Schedina */
            color: white;
            font-weight: bold;
        }
        .schedina-table tr:nth-child(even){background-color: #f2f2f2;}
        .schedina-table tr:hover {background-color: #ddd;}
        .team-home { font-weight: bold; text-align: right; }
        .team-away { font-weight: bold; text-align: left; }
        .prediction-cell { font-size: 1.2em; font-weight: bold; }
        .confidence-bar {
            background-color: #e0e0e0;
            border-radius: 3px;
            height: 18px;
            width: 100%;
            position: relative;
        }
        .confidence-fill {
            background-color: #4CAF50;
            height: 100%;
            border-radius: 3px;
            position: absolute;
            left: 0;
            top: 0;
            text-align: right;
            padding-right: 5px;
            color: white;
            font-size: 0.8em;
            line-height: 18px; /* Allinea verticalmente testo */
        }
         .prob-cell { font-size: 0.9em; color: #555; }
    </style>
    <table class='schedina-table'>
        <thead>
            <tr>
                <th>Casa</th>
                <th>Trasferta</th>
                <th>Esito</th>
                <th>Confidenza</th>
                <th>Prob. 1</th>
                <th>Prob. X</th>
                <th>Prob. 2</th>
            </tr>
        </thead>
        <tbody>
    """

    for res in prediction_results:
        confidence_pct = res['Confidence'] * 100
        confidence_color_hue = confidence_pct * 1.2  # Variazione di colore basata sulla confidenza
        html += f"""
            <tr>
                <td class='team-home'>{res['HomeTeam']}</td>
                <td class='team-away'>{res['AwayTeam']}</td>
                <td class='prediction-cell'>{res['Icon']} ({res['Prediction']})</td>
                <td>
                    <div class='confidence-bar'>
                        <div class='confidence-fill' style='width:{confidence_pct:.1f}%; background-color: hsl({confidence_color_hue}, 80%, 40%);'>
                           {confidence_pct:.1f}%
                        </div>
                    </div>
                </td>
                <td class='prob-cell'>{res['Prob_1']:.2f}</td>
                <td class='prob-cell'>{res['Prob_X']:.2f}</td>
                <td class='prob-cell'>{res['Prob_2']:.2f}</td>
            </tr>
        """

    html += "</tbody></table>"
    display(HTML(html))


# --- 8. ESECUZIONE PRINCIPALE ---
if __name__ == "__main__":
    # Carica i dati storici
    historical_data = load_data(DATA_DIRECTORY)

    if not historical_data.empty:
        # Allena l'agente Q-Learning
        train_q_learning(historical_data, TRAINING_EPISODES, LEARNING_RATE, DISCOUNT_FACTOR,
                         EXPLORATION_PROB_START, EXPLORATION_PROB_END, EXPLORATION_DECAY_RATE)

        # Definisci le partite per cui vuoi la previsione (ESEMPIO)
        upcoming_matches = [
            ("Inter", "Juventus"),
            ("Milan", "Roma"),
            ("Napoli", "Lazio"),
            ("Atalanta", "Fiorentina"),
            ("Bologna", "Torino"),
            ("Sassuolo", "Verona"),
            # Aggiungi altre partite...
        ]

        # Genera le previsioni per la "schedina"
        schedina_predictions = generate_schedina_predictions(upcoming_matches, historical_data, q_table)

        # Visualizza la schedina in formato HTML
        display_schedina_html(schedina_predictions)

        # (Opzionale) Visualizza in formato testo semplice con tabulate
        print("\n--- Schedina Previsioni (Testo) ---")
        table_data = [[p['HomeTeam'], p['AwayTeam'], p['Prediction'], f"{p['Confidence']:.2f}"] for p in
                      schedina_predictions]
        print(tabulate(table_data, headers=["Casa", "Trasferta", "Esito", "Confidenza"], tablefmt="grid"))

    else:
        print("Impossibile procedere senza dati storici.")